# Get started with Metrics Tracking

This notebook demonstrates how to use MLFlow to:
- Log metrics, params, and artifacts to MLFlow.
- Log, register, and load models using a local MLflow Tracking Server.
- Interact with the MLflow Tracking Server using the MLflow fluent API.
- Perform inference on Pandas DataFrames by loading models as generic Python Functions (pyfunc).

In [13]:
%load_ext autoreload
%autoreload 2

import joblib
import mlflow
from mlflow.models import infer_signature
import mlflow.sklearn
import pandas as pd
from pathlib import Path
from sklearn import ensemble, model_selection
from sklearn.metrics import mean_squared_error, mean_absolute_error

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Train model and calculate metrics

## Load Data

More information about the dataset can be found in UCI machine learning repository: https://archive.ics.uci.edu/ml/datasets/bike+sharing+dataset

Acknowledgement: Fanaee-T, Hadi, and Gama, Joao, 'Event labeling combining ensemble detectors and background knowledge', Progress in Artificial Intelligence (2013): pp. 1-15, Springer Berlin Heidelberg

In [14]:
# Download original dataset with: python src/load_data.py 

raw_data = pd.read_csv("../data/raw_data.csv")
raw_data.head()

,instant,dteday,season,yr,mnth,hr,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,casual,registered,cnt
0,1,2011-01-01,1,0,1,0,0,6,0,1,0.24,0.2879,0.81,0.0,3,13,16
1,2,2011-01-01,1,0,1,1,0,6,0,1,0.22,0.2727,0.80,0.0,8,32,40
2,3,2011-01-01,1,0,1,2,0,6,0,1,0.22,0.2727,0.80,0.0,5,27,32
3,4,2011-01-01,1,0,1,3,0,6,0,1,0.24,0.2879,0.75,0.0,3,10,13
4,5,2011-01-01,1,0,1,4,0,6,0,1,0.24,0.2879,0.75,0.0,0,1,1


## Prepare data

In [15]:
target = 'cnt'
prediction = 'prediction'
numerical_features = ['temp', 'atemp', 'hum', 'windspeed', 'mnth', 'hr', 'weekday']
categorical_features = ['season', 'holiday', 'workingday', ]

In [16]:
sample_data = raw_data.set_index('dteday').loc['2011-01-01 00:00:00':'2011-01-28 23:00:00'].reset_index()

X_train, X_test, y_train, y_test = model_selection.train_test_split(
    sample_data[numerical_features + categorical_features],
    sample_data[target],
    test_size=0.3
)

print(X_train.shape)
print(X_test.shape)

(415, 10)
(179, 10)


## Train a  Model

In [17]:
model = ensemble.RandomForestRegressor(random_state = 0, n_estimators = 50)
model.fit(X_train, y_train) 

model_path = Path('../models/model.joblib')
joblib.dump(model, model_path)

['../models/model.joblib']

In [18]:
model

RandomForestRegressor(n_estimators=50, random_state=0)

## Calculate Metrics

In [19]:
from sklearn.metrics import mean_squared_error, mean_absolute_error

preds = model.predict(X_test)

me = mean_squared_error(y_test, preds)
mae = mean_absolute_error(y_test, preds)

print(me, mae)

242.38987709497206 10.724692737430166


# Metrics Tracking with MLflow

## Set up MLFlow

In [8]:
MLFLOW_TRACKING_URI = "http://localhost:5001"

mlflow.set_tracking_uri(MLFLOW_TRACKING_URI)


## Log params, metrics and artifacts

In [9]:
with mlflow.start_run() as run: 

    # Log params 
    mlflow.log_param('model', 'RandomForest') 
    mlflow.log_params({'random_state': 0, 'n_estimators': 50})

    # Log metrics
    mlflow.log_metric('me', round(me, 3))
    mlflow.log_metric('mae', round(mae, 3))

    # Log the sklearn model and register as version 1
    mlflow.log_artifact("../data/raw_data.csv")
    mlflow.log_artifact("../models/model.joblib")


    # Set a tag that we can use to remind ourselves what this run was for
    mlflow.set_tag("random-forest", "Random Forest Classifier")

    # Infer the model signature
    signature = infer_signature(X_train, model.predict(X_train))

    # Log the model
    model_info = mlflow.sklearn.log_model(
        sk_model=model,
        artifact_path="rf_model",
        signature=signature,
        input_example=X_train,
        registered_model_name="1-get-started-random-forest",
    )

/Users/mikhailrozhkov/.pyenv/versions/3.11.6/lib/python3.11/site-packages/mlflow/types/utils.py:407: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/docs/latest/models.html#handling-integers-with-missing-values>`_ for more details.
  warnings.warn(
Registered model '1-get-started-random-forest' already exists. Creating a new version of this model...
2024/10/17 20:23:18 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model versio

2024/10/17 20:23:18 INFO mlflow.tracking._tracking_service.client: 🏃 View run useful-calf-997 at: http://localhost:5001/#/experiments/0/runs/1c0ee65e719445fbb7c53027741cf66d.
2024/10/17 20:23:18 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: http://localhost:5001/#/experiments/0.


## Load our saved model as a Python Function

Although we can load our model back as a native scikit-learn format with `mlflow.sklearn.load_model()`, below we are loading the model as a generic Python Function, which is how this model would be loaded for online model serving. We can still use the `pyfunc` representation for batch use cases, though, as is shown below.

In [10]:
model_info.model_uri

'runs:/1c0ee65e719445fbb7c53027741cf66d/rf_model'

In [11]:
loaded_model = mlflow.pyfunc.load_model(model_info.model_uri)

loaded_model

mlflow.pyfunc.loaded_model:
  artifact_path: rf_model
  flavor: mlflow.sklearn
  run_id: 1c0ee65e719445fbb7c53027741cf66d

## Use `loaded_model` to get predictions for `X_test` dataset

In [12]:
predictions = loaded_model.predict(X_test)

# Convert X_test validation feature data to a Pandas DataFrame
result = pd.DataFrame(X_test)

# Add the actual classes to the DataFrame
result["actual_class"] = y_test

# Add the model predictions to the DataFrame
result["predicted_class"] = predictions

result[:4]

,temp,atemp,hum,windspeed,mnth,hr,weekday,season,holiday,workingday,actual_class,predicted_class
148,0.20,0.1818,0.69,0.3881,1,11,6,1,0,0,62,70.86
500,0.06,0.0606,0.41,0.2239,1,23,0,1,0,0,21,23.92
328,0.26,0.2576,0.56,0.1642,1,4,0,1,0,0,1,2.38
110,0.20,0.2121,0.51,0.1642,1,20,4,1,0,1,69,61.78
